# Workflow Tutorial for a GOES-R (Geostationary) ABI Level 2 Data File

This tutorial was written in February 2023 by Dr. Amy Huff, IMSG at NOAA/NESDIS/STAR (amy.huff@noaa.gov). It demonstrates a practical example of how to use the **Xarray**, **netCDF4**, **NumPy**, **Matplotlib**, and **Cartopy** Python packages to work with an ABI Level 2 file in netCDF4 (.nc) format.  

The main workflow steps are:
- Open the ABI .nc file
- Read the file metadata
    - Read global metadata and metadata for data variables
    - Recognize when data variables have a "scale factor" and "add offset" applied
    - Recognize GOES-R ABI fixed grid projection coordinates
- Visualize pixel data on a map
    - Select a color map for the data & add a color bar to the plot
    - Use the native geostationary projection
    - Use other map projections
    - Format a map plot title automatically using information from satellite file name
- Save map image file

## Step 0: Read data product documentation

Before working with a new dataset, it is **imperative to find and read the documentation for the data**. Documentation contains important information on how to work correctly with a dataset, such as the valid data range, spatial coverage, spatial and temporal resolution, data quality flags, etc.

In this tutorial, we will be working with a **GOES-16 ABI Aerosol Optical Depth (AOD)** data file. 
- GOES-16 (also called GOES-East) is a geostationary satellite, part of NOAA's GOES-R series
- ABI is the Advanced Baseline Imager, a sensor on the GOES-16, GOES-17, and GOES-18 satellites
- ABI AOD is a Level 2 product, derived from ABI L1b data (radiances)
    - AOD is a quantitative measure of aerosols (e.g., smoke, dust, haze) in a vertical column of the atmosphere
    - ABI AOD data are available for the Full Disk and CONUS [scan sectors](https://www.star.nesdis.noaa.gov/atmospheric-composition-training/satellite_data_abi_scan_sectors.php)
    - ABI AOD data have 2 km spatial resolution and are available every 5 min (CONUS) or 10 min (Full Disk) during daylight

ABI AOD documentation:
- [GOES-16 ReadMe for Data Users](https://www.star.nesdis.noaa.gov/atmospheric-composition-training/documents/GOES-16_ABI_L2_AOD_Provisional_ReadMe_v3.pdf)
- [Algorithm Theoretical Basis Document (ATBD)](https://www.star.nesdis.noaa.gov/atmospheric-composition-training/documents/ABI_AOD_ATBD_V4.2_20180214.pdf)

## Step 1: Import Python packages

Four Python packages (libraries) and two Python modules are used in this Notebook:
- The **Xarray** library is used to work with labeled multi-dimensional arrays
- The **NumPy** library is used to perform array operations
- The **Matplotlib** library is used to make plots
- The **Cartopy** library is used to create maps
- The **datetime** module is used to manipulate dates and times
- The **pathlib** module is used to set filesystem paths for the user's operating system

In [ ]:
import xarray as xr

import numpy as np

import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker

from cartopy import crs as ccrs
import cartopy.feature as cfeature

import datetime

from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

## Step 2: Set the directory path for the satellite data file

It is good practice to set directory paths using the [pathlib module](https://docs.python.org/3/library/pathlib.html#module-pathlib), which automatically uses the correct format for the user's operating system. This helps avoid errors in situations when more than one person is using the same code file, because Windows uses back slashes in directory paths, while MacOS and Linux use forward slashes.

For this tutorial, the ABI data file is located in a directory on the training server.  The same **pathlib** syntax (```Path('directory_name')```) can be used to set the path for any directory, including a directory located on a local computer or a remote server.

In [ ]:
directory_name = '/tljh-data/sat_data/NOAA'
directory_path = Path(directory_name)

## Step 3: Open a GOES-16 ABI AOD satellite data file using Xarray

Let's open the GOES-16 ABI Aerosol Optical Depth (AOD) CONUS sector satellite data file (```file_name```). The full path for the data file (```file_id```) is set using **pathlib** syntax.

To open a single satellite data file with **Xarray**, use ```xr.open_dataset()```. The contents of a data file are called a "Dataset" in **Xarray**, conventionally abbreviated as ```ds```. 

In [ ]:
file_name = 'OR_ABI-L2-AODC-M6_G16_s20222591501173_e20222591503546_c20222591506172.nc'
file_id = directory_path / file_name

ds = xr.open_dataset(file_id, engine='netcdf4')

### Step 3.1: Print the file metadata using Xarray

Running the name of the Dataset (```ds```) will print the file metadata.

The global file metadata are listed under ```Attributes```. We can see these data come from the GOES-16 (GOES-East) satellite and the CONUS sector, and the data have 2 km spatial resolution at nadir.

For any of the ```Data variables``` or ```Coordinates```, click on the attributes icon to see the metadata and and click on the data repository icon to see a summary (excerpt) of the values in the array.  In this tutorial, we will be working with the ```AOD```, ```DQF```, and ```goes_imager_projection``` ```Data variables``` and the ```y``` and ```x``` ```Coordinates```.

In [ ]:
ds

### Step 3.2: Open the AOD data array as a NumPy array to see values

The satellite data in the file are displayed under ```Data variables```.  A data variable is called a "DataArray" in **xarray**, conventionally abbreviated as ```da```.

The data arrays in this file are too long to be displayed by the data repository icon in the ```Data variables``` metadata. When working with a new data file, it's good practice to take a look at the data arrays before proceeding to analyze or visualize the data.

Let's use the ```xr.DataArray.values``` function to open the ```AOD``` data array as a **NumPy** array, and print the array so we can see excerpts of the values. 

In [ ]:
ds.AOD.values

### Step 3.3: Print maximum and minimum  values in the AOD array, ignoring any NaNs, using NumPy

There are missing or invalid data at the beginning and end of the AOD array, which are displayed as "nan". To see the range of valid data in the array, we can use **NumPy** to print the non-NaN maximum and minimum values. Note these values are floating point numbers (floats).

In [ ]:
np.nanmin(ds.AOD.values), np.nanmax(ds.AOD.values)

### Step 3.4: Compare max/min of AOD data array to valid range in metadata

When working with a new dataset, it's good practice to compare the max/min of data in the data array to the valid range given in the metadata.  But when we do that for the AOD data, we see there is a discrepancy in the **data type**: the max/min are returned as floating point numbers (float) but the valid range is unsigned 16-bit integers (uint16). 

The AOD data are stored in the netCDF4 data file as unsigned integers, in order to save file space.  NetCDF files follow conventions for packing and unpacking data using the ```scale_factor``` and ```add_offset``` attributes. If the argument ```decode_cf=True``` (default) is used with ```xr.open_dataset()```, **Xarray** will automatically decode the values in the data arrays, meaning it will automatically apply the ```scale_factor``` and ```add_offset``` and convert the stored AOD values from integers to floats.

In [ ]:
ds.AOD.valid_range

### Step 3.5: Convert valid range stored as unsigned integers to floats

Let's manually convert the valid range in the metadata to floating point numbers, so we can see the actual range of valid AOD values.

```xarray.DataArray.encoding``` lists the attributes used to unpack data arrays. Multiplying the ```valid_range``` by the ```scale_factor``` and adding the ```add_offset``` gives the valid range of the AOD data as [-0.05, 5]. Note that the upper value returned is not exactly "5" because [Python has some inconsistencies in how it displays floats](https://docs.python.org/3/tutorial/floatingpoint.html#tut-fp-issues). The max/min values of the AOD data array returned in Step 3.3 fall within the valid range.

In [ ]:
ds.AOD.encoding

In [ ]:
ds.AOD.valid_range * ds.AOD.encoding['scale_factor'] + ds.AOD.encoding['add_offset']

## <span style='color:blue'>Exercise 1: Open the DQF data array as a NumPy array</span>

In the cell below, print the values of the "DQF" data array.

What is the "long name" of the "DQF" data array? What are the values and meanings of the data in the array? 

How do you think we can use the "DQF" data array in conjunction with the "AOD" data array?

In [ ]:
# Exercise 1: Open the "DQF" data array as a NumPy array to see the values



<details><summary><b><font color="blue">Click here for the solution to Exercise 1</font></b></summary>
    <p></p>

<div style="background: #f8f8f8;overflow:auto;width:auto;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">ds<span style="color: #666666">.</span>DQF<span style="color: #666666">.</span>values
</pre></div>

</details>

## Step 4: Make a quick plot of AOD using Xarray

When working with a new data product, it's good practice to make a quick plot to check the distribution of data in the file. An easy way to do this is by using [Xarray's plotting functions](https://docs.xarray.dev/en/stable/user-guide/plotting.html), which are built on top of **Matplotlib**.

Since the AOD data are pixel data, we can use **Xarray's** default [pcolormesh plot](https://docs.xarray.dev/en/stable/generated/xarray.plot.pcolormesh.html#xarray.plot.pcolormesh). The [Xarray 2-dimensional simple plotting tutorial](https://docs.xarray.dev/en/stable/user-guide/plotting.html#id2) has some useful examples as a reference.

Let's make a simple plot of "top 2 qualities" AOD, which includes the "high" and "medium" quality AOD data, selected using the ```xr.DataArray.where()``` [function](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.where.html) and the DQF data array.  The NOAA Aerosols and Atmospheric Composition Science Team recommends using "top 2" qualities AOD data for qualitative applications, such as forecasting and monitoring aerosols, and "high" quality data for quantitative applications, such as data assimilation.

In [ ]:
ds.AOD.where(ds.DQF <= 1).plot()

### Step 4.1: Select an appropriate color map

The plot generated using the **Xarray** default settings looks horrible. It's very difficult to interpret the AOD data because the default colormap is a "diverging" colormap, but we know the AOD valid range includes almost all positive values.  So let's change the colormap to a "sequential" colormap, used to represent information that has ordering.

**Xarray's** plotting functions are built on top of **Matplotlib**, so we can select a colormap from the [Matplotlib's standard colormaps](https://matplotlib.org/stable/tutorials/colors/colormaps.html).

Note that adding "_r" to the end of any colormap name reverses it (e.g., "plasma_r")

In [ ]:
cmap = plt.get_cmap('rainbow')
ds.AOD.where(ds.DQF <= 1).plot(cmap=cmap)

### Step 4.2: Change the plotted data range & set a unique color for extreme values

Even with a diverging colormap, it's still difficult to discern any pattern in the plotted AOD data because most values fall in the range of 0 to 1 (green shading). We want to emphasize thick aerosols, AOD > 1. Also, the low end of the default data range extends to -5 but we know the low end of the AOD valid range is -0.05.

So let's set the plotted AOD data range as [-0.05, 1], and set a unique color for plotted AOD > 1 using [Matplotlib's standard named colors](https://matplotlib.org/stable/gallery/color/named_colors.html). 

Now the areas with high concentrations of aerosols are evident: on this day, smoke, transported from large wildfires in the Western US, had reached the Eastern US.

In [ ]:
cmap = plt.get_cmap('rainbow').with_extremes(over='darkred')
ds.AOD.where(ds.DQF <= 1).plot(cmap=cmap, vmin=-0.05, vmax=1)

## <span style='color:blue'>Exercise 2: Select your own colormap and unique color for AOD > 1</span>

In the cell below, fill in the missing strings in ```get_cmap('')``` (colormap) and ```(over='')``` (color for AOD > 1) with your choices and generate the plot of AOD. 

Try a few different options to see the effects on the ease of readability/interpretation of the plotted data.

In [ ]:
# Exercise 2: Select your own colormap and color for AOD > 1

cmap = plt.get_cmap('').with_extremes(over='')
ds.AOD.where(ds.DQF <= 1).plot(cmap=cmap, vmin=-0.05, vmax=1)

## Step 5: Import Latitude and Longitude data arrays for GOES-16 ABI CONUS fixed grid


You may have noticed that there are no arrays for latitude or longitude in the ```Coordinates```.  To save file space, ABI Level 1b (radiances) and most Level 2 (derived) data files do not contain latitude and longitude. Instead, the data files contain information about the [ABI fixed grid](https://www.star.nesdis.noaa.gov/atmospheric-composition-training/satellite_data_goes_imager_projection.php), including ```y``` and ```x``` (GOES fixed grid projection y- and x-coordinates) and the ```goes_imager_projection``` (GOES-R ABI fixed grid projection constants), which can be used to calculate latitude and longitude.

Because the ABI CONUS and Full Disk sector domains are fixed for a given GOES-R satellite (they don't vary by date or time of day), latitude and longitude can be calculated once offline and the arrays saved in a separate netCDF file. Then whenever we want to plot an ABI Level 2 data array, such as AOD, we can open the latitude-longitude file and read the variable arrays.

You may be wondering why we can't plot AOD using the ```y``` and ```x``` ```Coordinates```, like we did in Step 4. That is fine for a quick plot to check the data, but it won't work for a plot of the ```AOD``` data array on a map projection, because the ```AOD``` array is 2-dimensional with dimensions of (```y```, ```x```) and the ```y``` and ```x``` arrays are 1-dimensional.

In [ ]:
lat_lon_file_name = 'g16_conus_lat_lon.nc'
lat_lon_file_id = directory_path / lat_lon_file_name

ds_lat_lon = xr.open_dataset(lat_lon_file_id, engine='netcdf4')

In [ ]:
ds_lat_lon

## <span style='color:blue'>Exercise 3: Open the Latitude and Longitude data arrays as a NumPy arrays</span>

In the cells below, print the values of the "Latitude" and "Longitude" data arrays.

In [ ]:
# Exercise 3: Open the "Latitude" data array as a NumPy array to see the values



In [ ]:
# Exercise 3: Open the "Longitude" data array as a NumPy array to see the values



<details><summary><b><font color="blue">Click here for the solution to Exercise 3</font></b></summary>
    <p></p>

<div style="background: #f8f8f8; overflow:auto;width:auto;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">ds_lat_lon<span style="color: #666666">.</span>Latitude<span style="color: #666666">.</span>values
</pre></div>
    
<div style="background: #f8f8f8; overflow:auto;width:auto;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">ds_lat_lon<span style="color: #666666">.</span>Longitude<span style="color: #666666">.</span>values
</pre></div>

</details>

## Step 6: Define the native geostationary map projection

We will use **Cartopy** to set the map projection for our plot of AOD. **Cartopy** has many different [map projection options](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html), each with its own strengths and limitations. Choose a map projection that highlights/emphasizes the satellite data with which you are working.

A popular choice for ABI data is the native geostationary projection of the satellite - GOES-16 in this case. We can use the information in the ```goes_imager_projection``` ```Data variable``` to define a [geostationary projection](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#geostationary) using **Cartopy**.

In [ ]:
projection_variables = ds.goes_imager_projection
satellite_height = projection_variables.perspective_point_height
semi_major_axis = projection_variables.semi_major_axis
semi_minor_axis = projection_variables.semi_minor_axis
central_longitude = projection_variables.longitude_of_projection_origin

globe = ccrs.Globe(semimajor_axis=semi_major_axis, semiminor_axis=semi_minor_axis)
geo_projection = ccrs.Geostationary(central_longitude=central_longitude, satellite_height=satellite_height,
                                    globe=globe, sweep_axis='x')

## Step 7: Define "levels" array for filled contour plot of AOD

For our plot of AOD data, we will use the **Matplotlib** ```plt.contourf``` [function](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contourf.html), which plots filled contours. Before we proceed to the plotting steps, we need to define an array that contains the specified levels at which the contour lines will be drawn.

We use **NumPy's** ```np.arange(start, stop, step)``` [function](https://numpy.org/doc/stable/reference/generated/numpy.arange.html) to create the ```contours_array``` with the AOD data range for the plot as [0,1] and a suitable contour interval (e.g., 0.05). The smaller the contour interval, the more detailed the plot will be. 

Note: we set the lower boundary of the plotted range as 0 instead of -0.05 because small negative values of AOD indicate the uncertainty in the AOD retrieval and should be considered as very small positive numbers, so we will plot them with the same color as for AOD=0.

In [ ]:
contour_interval = 0.05
contours_array = np.arange(0, 1+contour_interval, contour_interval)

In [ ]:
contours_array

## Step 8: Plot "top 2" qualities AOD on the native geostationary map projection using Matplotlib & Cartopy

We set up a figure in **Matplotlib** and add ```geoaxes``` with **Cartopy** using the geostationary map projection we defined in Step 6. 

We set the domain of the map to be the Continental US (CONUS) using **Cartopy's** ```set_extent()``` [function](https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.mpl.geoaxes.GeoAxes.html#cartopy.mpl.geoaxes.GeoAxes.set_extent) and add latitude/longitude gridlines and grid labels using **Cartopy's** ```gridlines``` [function](https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.mpl.geoaxes.GeoAxes.html#cartopy.mpl.geoaxes.GeoAxes.gridlines). The **Cartopy** documentation has a [short tutorial](https://scitools.org.uk/cartopy/docs/latest/matplotlib/gridliner.html) that shows a couple of examples of adding gridlines and labels to a map, for reference.

**Cartopy** has a number of options for adding coastlines and borders to a map. The simplest option is to use [Cartopy's Feature interface](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html), which defines seven common features at 1:110m (coarse) resolution, which is fine for our CONUS map. These coastlines/borders plot as black lines with linewidth=1 by default, but they can be modified by defining different colors or linewidths as arguments).

We plot the AOD data using **Matplotlib's** ```plt.contourf``` [function](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contourf.html), with the colormap and color for AOD > 1 that we defined in Step 4 and the ```contours_array``` defined in Step 7. Note **Xarray** makes it easy to plot Data Arrays using ```xr.DataArray```, e.g. ```ds_lat_lon.Longitude```, ```ds_lat_lon.Latitude```, ```ds.AOD.where(ds.DQF <= 1)```.

The plotting function argument ```transform=ccrs.PlateCarree()``` tells **Cartopy** that the AOD data are in geographic coordinates (lat/lon). This argument **must** be included when plotting satellite data that are in geographic coordinates, or the data will not plot correctly on the map projection.

We add a colorbar using **Matplotlib's** ```fig.colorbar()``` [function](https://matplotlib.org/stable/api/figure_api.html#matplotlib.figure.Figure.colorbar), making the orientation of the colorbar horizontal, setting the fraction of original axes to use for the colorbar (```fraction=0.2```), setting the fraction of padding between the colorbar and map axes (```pad=0.05```), setting the fraction of the size of the colorbar to the figure (```shrink=0.5```), and manually setting the tick marks. We also add a  title for the colorbar (```set_label```), bolded and in 8-point font using **Matplotlib's** [text settings](https://matplotlib.org/stable/tutorials/text/text_props.html), and add labels for the tick marks (```set_xticklabels```).

We also add a [plot title](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.title.html) that is automatically generated by extracting information from the data file name (```file_id.name```) using [```str.split()```](https://docs.python.org/3/library/stdtypes.html#str.split) and indexing. The [pathlib ```Pure_Path.name``` method](https://docs.python.org/3/library/pathlib.html#pathlib.PurePath.name) returns a string representing the final path component, which is used to obtain the ABI data file name as a string.

The plot title includes the observation date and time in UTC.  The 7-digit observation date from the data file name is converted from "YYYYJJJ" format, where "JJJ" stands for the 3-digit Julian day, into a more user-friendly format ("DD Mon YYYY") using the **datetime** module's [```strftime``` and ```strptime``` format codes](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes).

In [ ]:
fig = plt.figure(figsize=(8, 10))
ax = plt.axes(projection=geo_projection)

ax.set_extent([-115, -60, 14, 52], crs=ccrs.PlateCarree())

lon_ticks = [-120, -100, -80, -60]
lat_ticks = [20, 30, 40, 50]
gl = ax.gridlines(draw_labels=True, linewidth=0.25, color='silver')
gl.rotate_labels = None
gl.bottom_labels = None
gl.left_labels = None
gl.xlocator = ticker.FixedLocator(lon_ticks)
gl.ylocator = ticker.FixedLocator(lat_ticks)
gl.xlabel_style = {'size': 8}
gl.ylabel_style = {'size': 8}

ax.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=3)
ax.add_feature(cfeature.BORDERS, linewidth=0.5, zorder=3)
ax.add_feature(cfeature.LAKES, facecolor='lightgrey')
ax.add_feature(cfeature.STATES, linewidth=0.25, zorder=3)
ax.add_feature(cfeature.LAND, facecolor='grey')
ax.add_feature(cfeature.OCEAN, facecolor='lightgrey')

cmap = plt.get_cmap('rainbow').with_extremes(over='darkred')

plot = ax.contourf(ds_lat_lon.Longitude, ds_lat_lon.Latitude, ds.AOD.where(ds.DQF <= 1), contours_array, 
                   cmap=cmap, extend='both', zorder=2, transform=ccrs.PlateCarree(), transform_first=True)

cb = fig.colorbar(plot, orientation='horizontal', fraction=0.2, pad=0.05, shrink=0.5, ticks=[0, 0.25, 0.5, 0.75, 1])
cb.set_label(label='Aerosol Optical Depth at 550nm', size=8, weight='bold')
cb.ax.set_xticklabels(['0', '0.25', '0.50', '0.75', '1.0'])

julian = datetime.datetime.strptime(file_id.name.split('_')[3][1:8], '%Y%j').date()
observation_date = julian.strftime('%d %b %Y')
image_title = 'GOES-'+ file_id.name.split('_')[2][1:] + '/ABI Aerosol Optical Depth  ' + observation_date + ' ' + file_id.name.split('_')[3][8:10] + ':' + file_id.name.split('_')[3][10:12] + ' UTC'
plt.title(image_title, pad=10, size=8, weight='bold')

plt.show()

## Step 9: Plot "top 2" qualities AOD on the Plate Carree map projection using Matplotlib & Cartopy

Using **Cartopy**, you can plot the AOD data on any map projection you'd like. Let's try another popular choice, the [Plate Carree equidistant cylindrical (equirectangular) projection](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#platecarree).

The code below is exactly the same as in Step 8 except for the second line, where the Plate Carree projection is called instead of the geostationary projection we defined in Step 6: ```plt.axes(projection=ccrs.PlateCarree())```.

In [ ]:
fig = plt.figure(figsize=(8, 10))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent([-130, -58, 14, 52], crs=ccrs.PlateCarree())

lon_ticks = [-120, -100, -80, -60]
lat_ticks = [20, 30, 40, 50]
gl = ax.gridlines(draw_labels=True, linewidth=0.25, color='silver')
gl.rotate_labels = None
gl.top_labels = None
gl.right_labels = None
gl.xlocator = ticker.FixedLocator(lon_ticks)
gl.ylocator = ticker.FixedLocator(lat_ticks)
gl.xlabel_style = {'size': 8}
gl.ylabel_style = {'size': 8}

ax.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=3)
ax.add_feature(cfeature.BORDERS, linewidth=0.5, zorder=3)
ax.add_feature(cfeature.LAKES, facecolor='lightgrey')
ax.add_feature(cfeature.STATES, linewidth=0.25, zorder=3)
ax.add_feature(cfeature.LAND, facecolor='grey')
ax.add_feature(cfeature.OCEAN, facecolor='lightgrey')

cmap = plt.get_cmap('rainbow').with_extremes(over='darkred')

plot = ax.contourf(ds_lat_lon.Longitude, ds_lat_lon.Latitude, ds.AOD.where(ds.DQF <= 1), contours_array, 
                   cmap=cmap, extend='both', zorder=2, transform=ccrs.PlateCarree(), transform_first=True)

cb = fig.colorbar(plot, orientation='horizontal', fraction=0.2, pad=0.05, shrink=0.5, ticks=[0, 0.25, 0.5, 0.75, 1])
cb.set_label(label='Aerosol Optical Depth at 550nm', size=8, weight='bold')
cb.ax.set_xticklabels(['0', '0.25', '0.50', '0.75', '1.0'])

julian = datetime.datetime.strptime(file_id.name.split('_')[3][1:8], '%Y%j').date()
observation_date = julian.strftime('%d %b %Y')
image_title = 'GOES-'+ file_id.name.split('_')[2][1:] + '/ABI Aerosol Optical Depth  ' + observation_date + ' ' + file_id.name.split('_')[3][8:10] + ':' + file_id.name.split('_')[3][10:12] + ' UTC'
plt.title(image_title, pad=10, size=8, weight='bold')

plt.show()

## <span style='color:blue'>Exercise 4: Customize the plot of "top 2" qualities AOD</span>

In the cell below, make your own plot of "top 2" qualities AOD data.  Try changing one of more of the following:
- Color map and color for AOD > 1
- Colors for land and water shading
- Appearance of gridlines and grid labels
- Map projection
- Contour interval for plotting AOD

Hint: Start by copying the code from Step 8 or 9, and then modifying one or more sections to customize the plot.

In [ ]:
# Exercise 4: Customize the plot of "top 2" qualities AOD (color map, color for AOD > 1, map projection, gridlines, etc.)



## Step 10: Save the plot of "top 2" qualities AOD using Matplotlib

The last step is to save the image file of the AOD map so we can use the image in a research paper, presentation, website, or social media.

Using the same approach as for the plot title, a name for the saved image file (```saved_file_name```) is automatically generated by extracting information from the satellite data file name, including the observation date and time.

The map can be saved as an image file using **Matplotlib's** [```fig.savefig``` function](https://matplotlib.org/stable/api/figure_api.html#matplotlib.figure.Figure.savefig).
- Set the resolution of the saved image file using the ```dpi=``` argument. 
    - The higher the dpi, the higher the figure resolution, but the larger the file size and the longer it will take to save the file. 
- Set the background color for the plot using the ```facecolor``` argument.
- Set ```bbox_inches='tight'``` to minimize the bounding box around the figure (to zoom in "tight" on the plot).
- Specify the format of the saved file (```saved_file_format```). 
    - The default format is .png
    - File format options include .eps, .jpeg, .jpg, .pdf, .pgf, .png, .ps, .raw, .rgba, .svg, .svgz, .tif, .tiff

**Comments are included below in the full code to visualize ABI AOD data, to notate each step in the process.**

In [ ]:
# Set up figure in Matplotlib and add geoaxes & set map projection using Cartopy
fig = plt.figure(figsize=(8, 10))
ax = plt.axes(projection=geo_projection)

# Set domain of map
# Use negative values to indicate °W longitude, e.g., 100 °W = -100
# Use negative values to indicate °S latitude, e.g., 30 °S = -30
ax.set_extent([-115, -60, 14, 52], crs=ccrs.PlateCarree())

# Add lat/lon gridlines & labels
lon_ticks = [-120, -100, -80, -60]
lat_ticks = [20, 30, 40, 50]
gl = ax.gridlines(draw_labels=True, linewidth=0.25, color='silver')
gl.rotate_labels = None
gl.bottom_labels = None
gl.left_labels = None
gl.xlocator = ticker.FixedLocator(lon_ticks)
gl.ylocator = ticker.FixedLocator(lat_ticks)
gl.xlabel_style = {'size': 8}
gl.ylabel_style = {'size': 8}

# Add coastlines, borders, and water/land shading
ax.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=3)
ax.add_feature(cfeature.BORDERS, linewidth=0.5, zorder=3)
ax.add_feature(cfeature.LAKES, facecolor='lightgrey')
ax.add_feature(cfeature.STATES, linewidth=0.25, zorder=3)
ax.add_feature(cfeature.LAND, facecolor='grey')
ax.add_feature(cfeature.OCEAN, facecolor='lightgrey')

# Set colormap and unique color for AOD > 1
cmap = plt.get_cmap('rainbow').with_extremes(over='darkred')

# Plot AOD using filled contours
# "transform=ccrs.PlateCarree()" argument is required b/c data are in geographic coordinates
# "transform_first=True" speeds plotting process by calculating contours in projection space
# The "zorder=2" argument plots AOD data under coastlines/borders & over land/ocean/lakes polygons
plot = ax.contourf(ds_lat_lon.Longitude, ds_lat_lon.Latitude, ds.AOD.where(ds.DQF <= 1), contours_array, 
                   cmap=cmap, extend='both', zorder=2, transform=ccrs.PlateCarree(), transform_first=True)

# Add colorbar
cb = fig.colorbar(plot, orientation='horizontal', fraction=0.2, pad=0.05, shrink=0.5, ticks=[0, 0.25, 0.5, 0.75, 1])
cb.set_label(label='Aerosol Optical Depth at 550nm', size=8, weight='bold')
cb.ax.set_xticklabels(['0', '0.25', '0.50', '0.75', '1.0'])

# Create plot title automatically using information from file name
# Use datetime module to extract observation date & reformat
julian = datetime.datetime.strptime(file_id.name.split('_')[3][1:8], '%Y%j').date()
observation_date = julian.strftime('%d %b %Y')
# Put extracted/reformated strings together to make title
image_title = 'GOES-'+ file_id.name.split('_')[2][1:] + '/ABI Aerosol Optical Depth  ' + observation_date + ' ' + file_id.name.split('_')[3][8:10] + ':' + file_id.name.split('_')[3][10:12] + ' UTC'
# Add plot title
plt.title(image_title, pad=10, size=8, weight='bold')

# Show plot
plt.show()

# Save figure
# "bbox_inches=tight" sets a "tight" bounding box around saved image
saved_file_format = '.png'
save_date = julian.strftime('%Y%m%d')
saved_file_name = 'g16_abi_top2_aod_' + save_date + '_' + file_id.name.split('_')[3][8:12] + saved_file_format
fig.savefig(saved_file_name, facecolor='w', dpi=300, bbox_inches='tight')

# Close plot
plt.close()